<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cats.jpg" height="1200" width="1200"> 

# Мини-домашка 4: регрессия

На семинаре мы пытались спрогнозировать сколько лайков поставит студент первого курса в вышкинский паблик с мемасами. Наша итоговая модель получилась довольно хлипкой. Хотелось бы улучшить её. В этом домашнем задании именно этим вы и займётесь. По ходу кода ниже будет довольно много пустых строк. Нужнозаполнить их своим кодом. 

In [22]:
import pandas as pd               # уже знакомый вам пакет для работы с таблицами
import numpy as np                # смутно знакомый вам пакет для работы с матрицами
import matplotlib.pyplot as plt   # уже смутно знакомый вам пакет для картинок :3
import seaborn as sns             # ещё один пакет для картинок 
plt.style.use('ggplot')     # правильный (очень красивый) стиль у графиков

%matplotlib inline

## 1. Подготовка 

Ровно также как мы это делали на семинаре, подгружаем данные и делаем их предобработку. 

In [23]:
df = pd.read_csv('../data/vk_main.csv', sep='\t')   # подгружаем датасет

# варим таргет в виде логарифма 
df['ln_likes_memes'] = df['likes_memes'].apply(lambda w: np.log(w) if w > 0 else -np.log(-1*w + 1))

# выбрасываем из выборки все переменные, в которых слишком много пропусков
isnull = df.isnull().sum()
df = df[isnull[isnull <= 56].index]

# Выборка с переменными-счётчиками
variables_cnt = [item for item in df.columns if item[-3:] == 'cnt']

# Оставляем только информативные переменные (картинки можно посмотреть в семинарской тетрадке)
variables_cnt = [item for item in variables_cnt if len(df[item].unique()) > 10]

# логарифмируем счётчики
df[variables_cnt] = df[variables_cnt].apply(lambda w: np.log(w + 1))

# отбираем дамми - переменные 
variables_dummy = [item for item in df.columns if item[-5:] == 'dummy']

# средние, медианы и максимумы
variables_mean = [item for item in df.columns if item[-4:] == 'mean']
variables_median = [item for item in df.columns if item[-6:] == 'median']
variables_max = [item for item in df.columns if item[-3:] == 'max']

# все полезные переменные
useful_variables = variables_cnt + variables_dummy + variables_mean + \
                   variables_median + variables_max + ['friends_mail_from_course_pct']

# заполняем пропуски нулями
df[useful_variables] = df[useful_variables].fillna(0)
df.head()

,country,first_name,last_name,likes_memes,uid,male_dummy,facebook_dummy,instagram_dummy,skype_dummy,twitter_dummy,...,photo_repost_cnt,photo_repost_max,photo_repost_mean,photo_repost_median,photo_text_len_cnt,photo_ava_change_cnt,photo_text_url_len_cnt,friends_from_course_cnt,friends_mail_from_course_pct,ln_likes_memes
0,Россия,Артём,Еркин,0,181029517,1,0,0,0,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,2.833213,0.375000,-0.000000
1,Россия,Армен,Хачатрян,15,73703994,1,0,1,0,0,...,1.609438,1.0,0.571429,1.0,0.0,0.0,0.0,3.871201,0.531915,2.708050
2,Россия,Abbos,Akhmedov,11,467673028,1,0,0,0,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,3.295837,0.500000,2.397895
3,Россия,Адам,Триандафилиди,0,144910934,1,0,1,1,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,4.430817,0.518072,-0.000000
4,Казахстан,Айжан,Саят,9,166045266,0,0,0,0,0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,3.401197,0.344828,2.197225


Разделим выборку на тренировочную и тестовую.

In [24]:
from sklearn.model_selection import train_test_split  # функция для деления 

# делим в пропорции 80 к 20
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=42) 

__[0.5] Объясните зачем выборку разбивают на тренировочную и тестовую__ 

__Ответ:__ 

## 2. Константный прогноз 

Давайте воспроизведём константный прогноз.

In [44]:
# для удобства запишем в y то, что мы хотим спрогнозировать
y_train = df_train['ln_likes_memes'].get_values()
y_test = df_test['ln_likes_memes'].get_values()

# а в X то, по чему мы это будем делать.
X_train = df_train[useful_variables].get_values()
X_test = df_test[useful_variables].get_values()

__[0] Постройте константный прогноз ровно также, как мы делали это на семинаре.__

In [45]:
# Ваш красивый код 

y_mean = np.mean(y_train)
y_pred_naive = np.ones(len(y_test)) * y_mean

Теперь давайте посмотрим на тестовой выборке насколько константный прогноз получился хорошим. Будем использовать в качестве метрики качества $MAE$. Подгрузите метрику из `sklearn` и выясните качество работы константного прогноза на тестовой выборке. За вдохновением можно снова обратиться к семинару, но не вздумайте переписывать один в один функцию `print_metrics`. За это буду отнимать баллы.

__[0.5] Найдите MAE константного прогноза.__ 

In [46]:
# Ваш оригинальный код

from sklearn import metrics
metrics.mean_absolute_error(y_test, y_pred_naive)

1.3478818795857703

Отлично! Теперь у нас есть с чем сравнить более сложные модели. Давайте займёмся их строительством. 

## 3. Устойчивая к выбросам регрессия 

В самом конце семинара мы с вами сделали следущую процедуру: 

1. Посмотрели какие переменные сильнее всего коррелируют с таргетом, оставили только их
2. Оценили устойчивую к выбросам регрессию и выяснили, что она работает лучше константного прогноза

Давайте воспроизведём код для её оценки. 

In [47]:
# сначала посмотрим на чнегативные корреляции
corr_neg = df_train[useful_variables].corrwith(df_train['ln_likes_memes']).sort_values()
corr_neg[:10]

male_dummy                        -0.191419
mobile_phone_dummy                -0.143059
private_prof_dummy                -0.117873
home_phone_dummy                  -0.093994
twitter_dummy                     -0.081323
subscriptions_cnt                 -0.058777
can_write_private_message_dummy   -0.048310
wall_like_median                  -0.047799
wall_comment_median               -0.046382
photo_repost_max                  -0.043836
dtype: float64

In [48]:
# теперь на позитивные корреляции
corr_pos = df_train[useful_variables].corrwith(df_train['ln_likes_memes']).sort_values(ascending=False)
corr_pos[:10]

friends_from_course_cnt    0.191652
videos_cnt                 0.161514
wall_text_len_mean         0.159197
photo_like_cnt             0.156200
pages_cnt                  0.151808
wall_text_len_median       0.149763
friends_cnt                0.126809
photos_cnt                 0.124521
instagram_dummy            0.116639
wall_text_len_cnt          0.113156
dtype: float64

In [49]:
# оставляем в выборке только самые важные переменные 
best_variables = list(corr_pos[:10].index) + list(corr_neg[:10].index)

# Забираем в выборку лучших из лучших 
X_train = df_train[best_variables].get_values()
X_test = df_test[best_variables].get_values()

Подготовка к оцениванию модели тут заканчивается. Хорошо бы понять в ней каждую строчку кода, потому что сейчас вам надо будет написать что-то похожее самостоятельно. Оцениваем модель!

In [50]:
from sklearn.linear_model import HuberRegressor

# Объявили модель
model_huber = HuberRegressor()

# Обучили модель на тренировочной выборке 
model_huber.fit(X_train, y_train)

# Сделали прогнозы на тестовой выборке 
y_pred_huber_base = model_huber.predict(X_test)

__[0.5] Найдите качество модели и убедитесь, что оно лучше константного прогноза.__

In [51]:
# ваша строчка кода
metrics.mean_absolute_error(y_test, y_pred_huber_base)

1.3622612933778262

Попробуйте взять в качетве таргета непрологарифмирование значение лайков. Насколько сильно испортилась модель? Почему? 

In [52]:
from sklearn.linear_model import LinearRegression # подгрузили модель

# Объявили модель
model_regression = LinearRegression()

# Обучили модель на тренировочной выборке 
model_regression.fit(X_train, y_train)

# Сделали прогнозы на тестовой выборке 
y_pred_regr = model_regression.predict(X_test)

metrics.mean_absolute_error(y_test, y_pred_regr)

1.3702177735836998

## В дз: 

* __Подозрение:__ оч важны дамми-переменные. Странно, что наличие твиттера так сильно влияет на уменьшение лайков. Посмотрите как часто встречаются в выборке люди с твиттером. Это есть на дамми-гистограмме. Постройте её. Как думаете эта переменная правда важна? Почему? 

Попробуйте удалить из рассмотрения все дамми-переменные и оценить две регрессии: линейную и хубера. Какое у меоделей качество? Стало ли оно лучше? 

* Сформулируйте гипотезу о том, какие переменные стоит оставить или выкинуть из рассмотрения. Почему вы так считате? Оцените модель на подходящих на ваш взгляд переменных. Получилось ли улучшить качество прогнозирования? 

* Обратите внимание, что на картинке с прогнозами очень густой является область с нулевым числом лайков. Может ли быть проблема в том, что в данных есть особая нулевая точка, которую надо рассматривать отдельно и придумывать для неё свою модель? Попробуйте выкинуть из рассмотрения всех, кто поставил нулевое число лайков и построить модель только для положительного их количества. Для моделирования используйте список фичей, который вы придумали в предыдущем пункте. Дополнительно с помощью корреляции с таргетом отберите из этого списка только самые важные фичи.